In [ ]:
# Desktop Access and GUI Tools
import subprocess
import os
import time
import socket
from IPython.display import HTML, display

def get_desktop_url():
    """Get the desktop access URL for this container"""
    try:
        # For JupyterHub spawned containers
        base_url = os.environ.get('JUPYTERHUB_BASE_URL', '/')
        user = os.environ.get('JUPYTERHUB_USER', 'user')
        server_name = os.environ.get('JUPYTERHUB_SERVER_NAME', '')
        
        if server_name:
            desktop_url = f"{base_url}user/{user}/{server_name}/proxy/6080/"
        else:
            desktop_url = f"{base_url}user/{user}/proxy/6080/"
            
        return f"https://rosforge.com{desktop_url}"
    except:
        return "http://localhost:6080"  # fallback

def check_gui_services():
    """Check if GUI services are running"""
    services_status = {}
    
    # Check if port 6080 is listening
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex(('127.0.0.1', 6080))
        services_status['desktop'] = result == 0
        sock.close()
    except:
        services_status['desktop'] = False
    
    # Check display
    try:
        result = subprocess.run(['xdpyinfo'], env={'DISPLAY': ':1'}, capture_output=True)
        services_status['display'] = result.returncode == 0
    except:
        services_status['display'] = False
    
    return services_status

# Get desktop URL and check services
desktop_url = get_desktop_url()
services = check_gui_services()

# Display interface
status_color = "#4CAF50" if services.get('desktop', False) else "#f44336"
status_text = "Ready" if services.get('desktop', False) else "Not Ready"
display_status = "Working" if services.get('display', False) else "Not Working"
desktop_status = "Available" if services.get('desktop', False) else "Not Available"

html_content = f"""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 25px; border-radius: 15px; margin: 20px 0;">
    <h1 style="margin: 0 0 20px 0;">🖥️ ROS2 Desktop Environment</h1>
    
    <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px; margin-bottom: 20px;">
        <h3>Desktop Status: <span style="color: {status_color}">🔴 {status_text}</span></h3>
        <p>Display Server: {'✅ ' + display_status}</p>
        <p>Web Interface: {'✅ ' + desktop_status}</p>
    </div>
    
    <div style="text-align: center;">
        <a href="{desktop_url}" target="_blank" 
           style="background: #4CAF50; color: white; padding: 15px 30px; text-decoration: none; border-radius: 8px; font-size: 18px; font-weight: bold; display: inline-block; margin: 10px;">
            🚀 Open Desktop Environment
        </a>
    </div>
    
    <div style="font-size: 14px; opacity: 0.8; text-align: center; margin-top: 15px;">
        URL: <code>{desktop_url}</code>
    </div>
</div>
"""

display(HTML(html_content))

print("🎮 Available GUI Applications:")
print("• RViz2 - 3D visualization: rviz2")
print("• Gazebo - Robot simulation: gz sim")
print("• TurtleSim - Learning tool: ros2 run turtlesim turtlesim_node")
print("• RQT Tools - Debugging: rqt")
print("• Terminal - Command line: xterm")

# Debug info
print(f"\n🔧 Debug Info:")
print(f"JUPYTERHUB_USER: {os.environ.get('JUPYTERHUB_USER', 'Not set')}")
print(f"JUPYTERHUB_BASE_URL: {os.environ.get('JUPYTERHUB_BASE_URL', 'Not set')}")
print(f"Desktop URL: {desktop_url}")